In [10]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
   text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [11]:
# tokenize the chars
string_to_int = { ch: i for i, ch in enumerate(chars)} # create a dictionary of the characters
int_to_string = {i:ch for i, ch in enumerate(chars)} # create a reverse dictionary of the characters
encode = lambda words: [string_to_int[char] for char in words]
decode = lambda chars: ''.join([int_to_string[char] for char in chars])

encode_hello = encode('hello')
print(encode_hello)
decoded_hello = decode(encode_hello)
print(decoded_hello)


[61, 58, 65, 65, 68]
hello


In [12]:
import torch

### Using batches to accelerate training process


In [13]:
# check if cuda is available in our hardware
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8 # length of the sequence
batch_size = 4 # number of blocks we are processing in parallel
max_iters = 10000 # number of iterations
learning_rate = 3e-4 # learning rate equivalent to 0.0003
eval_iters = 250
n_embd = 384 # number of embeddings
n_layer = 4 # number of layers

cpu


In [14]:
data = torch.tensor(encode(text), dtype=torch.long) # convert the text to a tensor
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


## Validation and Training Split


In [15]:
train_size = int(0.8 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print("inputs:")
# print(x.shape)
print(x)
print("target:")
print(y)

inputs:
tensor([[67, 78,  1, 73, 58, 58, 73, 61],
        [73, 61, 78,  9,  1, 56, 54, 73],
        [58, 66, 58, 57,  1, 73, 68,  1],
        [69, 68, 62, 67, 73,  1, 76, 54]])
target:
tensor([[78,  1, 73, 58, 58, 73, 61,  0],
        [61, 78,  9,  1, 56, 54, 73, 56],
        [66, 58, 57,  1, 73, 68,  1, 69],
        [68, 62, 67, 73,  1, 76, 54, 72]])


In [16]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y  = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [17]:
import torch.nn as nn
from torch.nn import functional as F 

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embeddings_table = nn.Embedding(vocab_size, n_embd)
        self.positional_embeddings_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer form
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, index, targets = None):
        logits = self.token_embeddings_table(index)
        
        # idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embeddings_table(index) # (B, T, C)
        pos_emb = self.positional_embeddings_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) 
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the functionos
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # becomes (B,C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # becomes (B,1)
            # append sampled index to the running sequence
            index = torch.cat([index, index_next], dim=1) # becomes (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


qym﻿z4yPuH!8S;SK*﻿&29LoM*LnL?UZo!"QdXV_I!D)invqhZ4
aZ5ty"egFL&m﻿nH7FAS(B!PA*OMCihn8XI:Y)eihKyLC9RfVz38_g.Yq.BX9q9CiiZxh8)'MS'Dhx0uQa_7 a3g:-auKBFpc﻿﻿sRjH3i0)yqVkRtW43VdJwYKl3IRp)y&fyzxznrol_UAFpS2]-(73DDxJ!P
,W8jano1_gCRlO?hr:AIU4&W&S[fDc2eAxqHS-9fI:ii7Cl*sDF2E]P:pAfolBxT8mxU-xn﻿(aPOffQRzkGQb !f3Hc. 4T8S-"URa2HZ'QV42cCRYKe2ck.vhsN-gDH7qyB*blpQ;Trm_gCR?wRyE0VyKXro43N!sR5.po_gZ.]"QN&-xUTd__)ShEIpa3w we2PNsR2EC9JARzxLNq7urJtIA)9 am!-hvXARe2xo!-ut(3)y[EmdSc[O(
nJIm*?QGMvi'g-1LgZ(Tr&e8JC9Cpa_)yY7_(N6


In [32]:
# create a PyTorch optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, test loss: {losses['test']:.4f}")
    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the  loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


step: 0, train loss: 2.4226, test loss: 2.4909
step: 250, train loss: 2.4314, test loss: 2.5095
step: 500, train loss: 2.4075, test loss: 2.4975
step: 750, train loss: 2.4081, test loss: 2.4770
step: 1000, train loss: 2.4202, test loss: 2.5006
step: 1250, train loss: 2.4355, test loss: 2.4934
step: 1500, train loss: 2.4263, test loss: 2.5046
step: 1750, train loss: 2.4293, test loss: 2.5127
step: 2000, train loss: 2.4553, test loss: 2.4489
step: 2250, train loss: 2.4395, test loss: 2.4569
step: 2500, train loss: 2.4504, test loss: 2.5075
step: 2750, train loss: 2.4487, test loss: 2.4802
step: 3000, train loss: 2.4566, test loss: 2.4526
step: 3250, train loss: 2.4437, test loss: 2.4746
step: 3500, train loss: 2.4396, test loss: 2.4823
step: 3750, train loss: 2.4275, test loss: 2.5069
step: 4000, train loss: 2.4383, test loss: 2.4804
step: 4250, train loss: 2.4453, test loss: 2.4777
step: 4500, train loss: 2.4343, test loss: 2.4992
step: 4750, train loss: 2.4194, test loss: 2.4874
step: 

***Need to familiarize ***
1. Mean Squared Error(MSE)
2. Gradient Descent (GD)
3. Momentum
4. RMSprop
5. Adam
6. AdamW

In [36]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

tensor([-0.0476])
